## IBM Watson Machine Learning using PMML

* 이 노트북은 Watson Machine learning 에서 sample 로 만들어져 저장되어 있는 PMML (Predictive Model Markup Language) 을 사용하여
* 꽃잎, 꽃받침에 대한 크기  데이터를 가지고 IRIS 품종을 예측하는 샘플입니다. 

## Learning goals

-  Watson Machine Learning (WML) instance를 생성하고
-  미리 만들어진 PMML model 불러서
-  test 데이터에 대한 scoring 을 수행합니다. 


## Contents

1.	[Set up the environment](#setup)
2.	[Work with the WML service instance](#instance) 
3.	[Persist, deploy and score in an IBM Cloud](#scoring)
4.	[Summary and next steps](#summary)


* 참고로 원본 소스에서 wget --> wget.download 함수로 수정되어야합니다. 

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a [Watson Machine Learning (WML) Service](https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/) instance (a free plan is offered and information about how to create the instance is [here](https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html))
-   **Not required on Watson Studio**: Install the [watson-machine-learning-client](https://pypi.python.org/pypi?%3Aaction=search&term=watson-machine-learning-client&submit=search)

<a id="instance"></a>
## 2. Working with the WML service instance

In this section you will learn how to get details about your WML service instance.

First, install the Python API client from pypi.

In [28]:
!rm -rf $PIP_BUILD/watson-machine-learning-client

In [30]:
!echo $PIP_BUILD
!ls -al pmml_sample_model



total 24
drwxr-x--- 2 dsxuser dsxuser  4096 Apr 16 10:38 .
drwxr-x--- 3 dsxuser dsxuser  4096 Apr 16 10:38 ..
-rw-r----- 1 dsxuser dsxuser 13876 Apr 16 10:38 iris_chaid.xml


In [31]:
#Install the WML client if required.
!pip install watson-machine-learning-client --upgrade

Requirement already up-to-date: watson-machine-learning-client in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement not upgraded as not directly required: ibm-cos-sdk in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement not upgraded as not directly required: tabulate in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement not upgraded as not directly required: certifi in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement not upgraded as not directly required: urllib3 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement not upgraded as not directly required: pandas in /opt/conda/envs/DSX-Pytho

Authenticate to the WML service on IBM Cloud.

**Tip**: Authentication information (your credentials) can be found in the [Service Credentials](https://console.bluemix.net/docs/services/service_credentials.html#service_credentials) tab of the service instance that you created on IBM Cloud. <BR>If you cannot see the **instance_id** field in **Service Credentials**, click **New credential (+)** to generate new authentication information. 

**Action**: Enter your WML service instance credentials here.

 ![WML Credentials](https://github.com/moreal70/IBM-Watson-Studio/raw/master/images/watson-ml-credential.png)

In [32]:
# Enter your credentials here.
wml_credentials = {
  "url": "https://ibm-watson-ml.mybluemix.net",
  "access_key": "j86cnGBYCcsTMsUFBCcgKBkjX5PqkAa/t9ma71APWHOEWHn65lGUX+iucyKk+LVaHxGxQ3pIogjgEOjN0TGDTcL0h32gVzPkwMbmHXNpi+FQYUqQmv73SQJrb1WXWeZv",
  "username": "5c4c9072-3613-4a1b-9eb9-e77d3c55c04a",
  "password": "899810ee-80d6-48bc-9567-2e5a401d4066",
  "instance_id": "0c748ad9-d4fc-441e-9b48-a6f5bf9e18d7"
}

In [33]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [34]:
wml_client = WatsonMachineLearningAPIClient(wml_credentials)

Details about your WML service instance are available in `instance_details`.

In [35]:
# List information about your WML service instance.
print(wml_client.instance_details)

{'entity': {'plan': 'lite', 'usage': {'prediction_count': {'limit': 5000, 'current': 1}, 'capacity_units': {'limit': 180000000, 'current': 0}, 'expiration_date': '2018-05-01T00:00:00.000Z', 'gpu_count': {'limit': 8, 'current': 0}, 'computation_time': {'limit': 180000, 'current': 0}, 'deployment_count': {'limit': 5, 'current': 1}, 'model_count': {'limit': 200, 'current': 2}}, 'source': 'Bluemix', 'organization_guid': '720d3e36-0ad6-4b64-a448-6b389465774f', 'region': 'us-south', 'published_models': {'url': 'https://ibm-watson-ml.mybluemix.net/v3/wml_instances/0c748ad9-d4fc-441e-9b48-a6f5bf9e18d7/published_models'}, 'space_guid': '0f23944d-88d2-48e0-ae74-d805e8855d9b', 'owner': {'email': 'parkhsu@kr.ibm.com', 'user_id': '062176d1-fd73-4425-b4bd-4d08d4c79721', 'beta_user': True, 'country_code': 'KOR', 'ibm_id': '120000M69P'}, 'plan_id': '3f6acf43-ede8-413a-ac69-f8af3bb0cbfe', 'status': 'Active', 'account': {'type': 'PAYG', 'id': '500115386', 'name': "hyeonsu Park's Account"}, 'deployments'

Get information about the stored models for your WML instance.

In [36]:
# List information about the stored models.
wml_client.repository.list_models()

------------------------------------  ---------------------------------------  ------------------------  -----------------
GUID                                  NAME                                     CREATED                   FRAMEWORK
694b337e-baa8-4f0c-b8c6-5741c9b6ea8e  PARKHSU created Digits prediction model  2018-03-29T02:37:53.795Z  scikit-learn-0.19
620fe809-1901-4487-b93c-9a500d4ccd30  CHAID PMML model for Iris data           2018-04-16T10:38:48.025Z  pmml-4.2
------------------------------------  ---------------------------------------  ------------------------  -----------------


Get a list of the deployed models for your WML service instance.

In [37]:
# List the deployed models.
wml_client.deployments.list()

------------------------------------  -----------------------  ------  --------------  ------------------------  ---------
GUID                                  NAME                     TYPE    STATE           CREATED                   FRAMEWORK
c57dbc5c-81db-4b36-94a0-08933935dd86  Iris species prediction  online  DEPLOY_SUCCESS  2018-04-16T10:39:25.363Z  pmml-4.2
------------------------------------  -----------------------  ------  --------------  ------------------------  ---------


<a id="scoring"></a>
## 3. Persist, deploy and score model in a Cloud

In this section you will use the Python API client for the WML service to create an online deployment and score a new data record.

For more information, see the <a href="http://wml-api-pyclient.mybluemix.net/" target="_blank" rel="noopener no referrer">Python API client documentation</a>.

- [3.1. Persist model](#persist)
- [3.2. Create the online deployment](#deploy)
- [3.3. Score data](#score)


### 3.1. Persist model<a id="persist"></a>

**Action**: Use `wget` to download the sample PMML model, `iris_chaid.xml` from the Git project.

In [38]:
!pip install wget

Requirement not upgraded as not directly required: wget in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages


In [40]:
# Download sample PMML model, iris_chaid.xml, from Git.
import wget
import os

* 예측모델을 저장할 directory를 생성하고 Watson machine learning 서비스가 제공하는 샘플 모델을 download 합니다.
* 여기서 원본 소소를  wget.download 로 수정해야 합니다. 

In [43]:
sample_dir = 'pmml_sample_model'
if not os.path.isdir(sample_dir):
    os.mkdir(sample_dir)
    
#filename=os.path.join(sample_dir, 'iris_chaid.xml')
#if not os.path.isfile(filename) :
filename = wget.download('https://github.com/pmservice/wml-sample-models/raw/master/pmml/iris-species/model/iris_chaid.xml', out=sample_dir)
filename

'pmml_sample_model/iris_chaid (1).xml'

Store the downloaded file as *CHAID PMML model for Iris data* in the WML service repository, then list all the files stored in the repository.

In [44]:
# Save the downloaded file to the WML repository.
props_pmml = {wml_client.repository.ModelMetaNames.NAME: 'CHAID PMML model for Iris data',
              wml_client.repository.ModelMetaNames.AUTHOR_NAME: "IBM", 
              wml_client.repository.ModelMetaNames.AUTHOR_EMAIL: "ibm@ibm.com", 
              wml_client.repository.ModelMetaNames.FRAMEWORK_NAME: "pmml", 
              wml_client.repository.ModelMetaNames.FRAMEWORK_VERSION: "4.2"}

model_details = wml_client.repository.store_model(filename, props_pmml)
wml_client.repository.list_models()

------------------------------------  ---------------------------------------  ------------------------  -----------------
GUID                                  NAME                                     CREATED                   FRAMEWORK
694b337e-baa8-4f0c-b8c6-5741c9b6ea8e  PARKHSU created Digits prediction model  2018-03-29T02:37:53.795Z  scikit-learn-0.19
620fe809-1901-4487-b93c-9a500d4ccd30  CHAID PMML model for Iris data           2018-04-16T10:38:48.025Z  pmml-4.2
f74a8ee1-09b6-4d89-856e-f4bab431a0ef  CHAID PMML model for Iris data           2018-04-16T11:13:01.858Z  pmml-4.2
------------------------------------  ---------------------------------------  ------------------------  -----------------


**Note:** From the list of downloaded files, you can see that model is successfully stored in the WML service repository.

### 3.2. Create the online deployment<a id="deploy"></a>

Now create the online deployment, *Iris species prediction*, for the stored model, then list all the online deployments for the model.

In [46]:
# Create the online deployment.
model_uid = wml_client.repository.get_model_uid(model_details)
model_deployment = wml_client.deployments.create(model_uid, name = 'Iris species prediction')



#######################################################################################

Synchronous deployment creation for uid: 'fe5e3c56-925f-4cdc-83a6-2b81da7cb421' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='fe5e3c56-925f-4cdc-83a6-2b81da7cb421'
------------------------------------------------------------------------------------------------




In [47]:
wml_client.deployments.list()

------------------------------------  -----------------------  ------  --------------  ------------------------  ---------
GUID                                  NAME                     TYPE    STATE           CREATED                   FRAMEWORK
7c61eb7b-d9b3-425d-be32-e97fcbbd2036  Iris species prediction  online  DEPLOY_SUCCESS  2018-04-16T11:13:49.463Z  pmml-4.2
c57dbc5c-81db-4b36-94a0-08933935dd86  Iris species prediction  online  DEPLOY_SUCCESS  2018-04-16T10:39:25.363Z  pmml-4.2
fe5e3c56-925f-4cdc-83a6-2b81da7cb421  Iris species prediction  online  DEPLOY_SUCCESS  2018-04-16T11:14:13.093Z  pmml-4.2
------------------------------------  -----------------------  ------  --------------  ------------------------  ---------


**Note:** From the list of deployed models, you can see that model was  successfully created.

### 3.3. Score data<a id="score"></a>

Send a sample record using the Python API client to check that the online scoring endpoint is available.

In [48]:
# Verify the online scoring endpoint.
model_scoring_url = wml_client.deployments.get_scoring_url(model_deployment)

print(model_scoring_url)

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/0c748ad9-d4fc-441e-9b48-a6f5bf9e18d7/published_models/f74a8ee1-09b6-4d89-856e-f4bab431a0ef/deployments/fe5e3c56-925f-4cdc-83a6-2b81da7cb421/online


You can see that the endpoint url is available for sending scoring requests.

Now, you can score data and predict the species of iris flower.

In [51]:
# Score data and predict species of flower.
scoring_data = {'fields': ['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width'], 'values': [[5.1, 3.5, 1.4, 0.2]]}
predictions = wml_client.deployments.score(model_scoring_url, scoring_data) 

print(model_scoring_url)
print(predictions)

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/0c748ad9-d4fc-441e-9b48-a6f5bf9e18d7/published_models/f74a8ee1-09b6-4d89-856e-f4bab431a0ef/deployments/fe5e3c56-925f-4cdc-83a6-2b81da7cb421/online
{'values': [['setosa', 1.0]], 'fields': ['PredictedValue', 'Confidence']}


As we can see from the prediction, this is the Iris Setosa flower.

<a id="summary"></a>
## 4. Summary and next steps     

You successfully completed this notebook! You learned how to use Watson Machine Learning for PMML model deployment and scoring. 

Check out our [documentation](https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html) for more samples, tutorials, documentation, how-tos, and blog posts. 

### Citations

Dua, D. and Karra Taniskidou, E. (2017). [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml). Irvine, CA: University of California, School of Information and Computer Science.


### Authors

**Wojciech Sobala** is a Data Scientist in IBM.

Copyright © 2018 IBM. This notebook and its source code are released under the terms of the MIT License.

<div style="background:#F5F7FA; height:100px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Want to do more?</span><span style="border: 1px solid #3d70b2;padding: 15px;float:right;margin-right:40px; color:#3d70b2; "><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
<span style="color:#5A6872;"> Try out this notebook with your free trial of IBM Watson Studio.</span>
</div>